# Day 10: Adapter Array

[*Advent of Code 2020 day 10*](https://adventofcode.com/2020/day/10) and [*solution megathread*](https://redd.it/ka8z8x)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2020/10/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2020%2F10%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
testdata1 = """16
10
15
5
1
11
7
19
6
12
4""".splitlines()

testdata2 = """28
33
18
42
31
14
46
20
48
47
24
23
49
45
19
38
39
11
1
32
25
35
8
17
7
9
4
2
34
10
3""".splitlines()

inputdata = downloaded['input'].splitlines()

No idea why I would choose the function name `analyze1`, but so it was...

In [6]:
def analyze1(adapters_sorted):
    differences = list(map(lambda t: t[1] - t[0],
                           zip([0] + adapters_sorted[:-1],
                               adapters_sorted + [adapters_sorted[-1] + 3])))
    # There's a hard-coded difference of 3 to the output joltage
    differences.append(3)
    distributions = dict()
    for d in range(1, 3 + 1):
        distributions[d] = differences.count(d)
    return(differences, distributions)

In [7]:
def print_adapters(adapters_sorted):
    print(f'(0), {", ".join([str(a) for a in adapters_sorted])}'
          f', ({adapters_sorted[-1] + 3})')

In [8]:
adapters_test1 = sorted(set([int(x) for x in testdata1]))
print_adapters(adapters_test1)
print(analyze1(adapters_test1))

(0), 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, (22)
([1, 3, 1, 1, 1, 3, 1, 1, 3, 1, 3, 3], {1: 7, 2: 0, 3: 5})


In [9]:
adapters_test2 = sorted(set([int(x) for x in testdata2]))
print_adapters(adapters_test2)
print(analyze1(adapters_test2))

(0), 1, 2, 3, 4, 7, 8, 9, 10, 11, 14, 17, 18, 19, 20, 23, 24, 25, 28, 31, 32, 33, 34, 35, 38, 39, 42, 45, 46, 47, 48, 49, (52)
([1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3, 1, 1, 3, 3, 1, 1, 1, 1, 3, 1, 3, 3, 1, 1, 1, 1, 3], {1: 22, 2: 0, 3: 10})


In [10]:
adapters = sorted(set([int(x) for x in inputdata]))
_, distributions = analyze1(adapters)
print(f'{distributions[1]}*{distributions[3]} = ' +
      f'{distributions[1]*distributions[3]}')

70*30 = 2100


In [11]:
HTML(downloaded['part1_footer'])

## Part Two

In [12]:
HTML(downloaded['part2'])

In the smaller example, it was possible to enumerate the various ways to combine adapters - though in the larger example and particularly the real input, that becomes unfeasible and one will have to settle to `count_the_ways`.
Apologies for using a global dict WAYS, I'd hopefully know better now...

In [13]:
def find_the_ways(low, adapters_sorted):
    if low in WAYS.keys():
        return WAYS[low]

    if len(adapters_sorted) == 1:
        return [adapters_sorted]

    ways_including = find_the_ways(adapters_sorted[0],
                                   adapters_sorted[1:])

    ways_excluding = []
    # If we can exclude adapters_sorted[0] and still progress
    if adapters_sorted[1] - low <= 3:
        if len(adapters_sorted) == 2:
            ways_excluding = [[adapters_sorted[1]]]
        else:
            ways_excluding = find_the_ways(low, adapters_sorted[1:])
    WAYS[low] = list(map(
            lambda w: [adapters_sorted[0]] + w, ways_including)) \
        + ways_excluding
    return WAYS[low]

In [14]:
WAYS = dict()
ways_test1 = find_the_ways(0, adapters_test1)
for w in ways_test1:
    print_adapters(w)
print(f'There are {len(ways_test1)} ways')

(0), 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, (22)
(0), 1, 4, 5, 6, 7, 10, 12, 15, 16, 19, (22)
(0), 1, 4, 5, 7, 10, 11, 12, 15, 16, 19, (22)
(0), 1, 4, 5, 7, 10, 12, 15, 16, 19, (22)
(0), 1, 4, 6, 7, 10, 11, 12, 15, 16, 19, (22)
(0), 1, 4, 6, 7, 10, 12, 15, 16, 19, (22)
(0), 1, 4, 7, 10, 11, 12, 15, 16, 19, (22)
(0), 1, 4, 7, 10, 12, 15, 16, 19, (22)
There are 8 ways


In [15]:
def count_the_ways(low, adapters_sorted):
    if low in WAYS.keys():
        return WAYS[low]
    if len(adapters_sorted) == 1:
        return 1
    ways_including = count_the_ways(adapters_sorted[0],
                                    adapters_sorted[1:])
    ways_excluding = 0
    # If we can exclude adapters_sorted[0] and still progress
    if adapters_sorted[1] - low <= 3:
        if len(adapters_sorted) == 2:
            ways_excluding = 1
        else:
            ways_excluding = count_the_ways(low, adapters_sorted[1:])
    WAYS[low] = ways_including + ways_excluding
    return WAYS[low]

In [16]:
WAYS = dict()
ways_count_test2 = count_the_ways(0, adapters_test2)
print(f'There are {ways_count_test2} ways')

There are 19208 ways


In [17]:
WAYS = dict()
ways_count = count_the_ways(0, adapters)
print(f'There are {ways_count} ways')

There are 16198260678656 ways


In [18]:
HTML(downloaded['part2_footer'])